In [ ]:
# Load Packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

import model_prep

datapath = "../data"

<h1 style="color:#27469c">Kissam Cooling Tower 1 - Summer Data</h1>

In [ ]:
step_back = 6 # window size = 6*5 = 30 mins

<h2 style="color:#27469c">1. Create timestepped data on pandas and normalize</h2>

In [ ]:
# load data
df = pd.read_csv(f'{datapath}/kissam/kissam_tower_1_preprocessed.csv', index_col='time')
df.index = pd.to_datetime(df.index)

# only take data for one season
df = model_prep.choose_season(df, season='summer', season_col_name='Kissam_Tower_1 season')

# save a boolean series that specifies whether the cooling tower is on
on_condition = df['Kissam_Tower_1 fanStatus']

# select features and targets and create final dataframe that includes only relevant features and targets
features = ['Kissam_Tower_1 enteringWaterTemp', 'outdoorAirDryBulb', 'outdoorAirWetBulb', 'Kissam_Tower_1 vfdPercent', 'Kissam_Tower_1 vfdPower', 'Kissam_Tower_1 CH_PowChi', 'Kissam_Tower_1 perFreqConP', 'Kissam_Tower_1 powConP', 'Kissam_Tower_1 CH_Tonnage']
targets = ['Kissam_Tower_1 leavingWaterTemp']
df = df[features].join(df[targets], on=df.index)

# normalize data
scaler = model_prep.NormalizationHandler()
df = scaler.normalize(dtframe=df, target_col=targets[0])

In [ ]:
df.columns

In [ ]:
# prepare dataframe for lstm by adding timesteps
lstm_df = model_prep.create_timesteps(df, n_in=step_back, n_out=1, target_names=targets)

# remove cases where spring data would leak into summer data (i.e. intial timesteps)
lstm_df = model_prep.remove_irrelevant_data(lstm_df, on_condition, step_back)

# save
lstm_df.to_csv(f"{datapath}/kissam/kissam1_summer_timestepped.csv")

<h2 style="color:#27469c">2. Split data into training and testing sets</h2>

In [ ]:
tss = TimeSeriesSplit(n_splits = 3)
X = lstm_df.drop([f'{target}(t)' for target in targets], axis=1) # drop target columns
y = lstm_df[[f'{target}(t)' for target in targets]] # only have target columns

for train_index, test_index in tss.split(X): # split into training and testing
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

<h2 style="color:#27469c">3. Get timestepped data as a 3D vector</h2>

In [ ]:
vec_X_train = model_prep.df_to_3d(lstm_dtframe=X_train, num_columns=len(features)+1, step_back=step_back)
vec_X_test = model_prep.df_to_3d(lstm_dtframe=X_test, num_columns=len(features)+1, step_back=step_back)

vec_y_train = y_train.values
vec_y_test = y_test.values

print(vec_X_train.shape, vec_X_test.shape, vec_y_train.shape, vec_y_test.shape)

<h2 style="color:#27469c">4. Create and Train model</h2>

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(vec_X_train.shape[1], vec_X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

history = model.fit(vec_X_train, vec_y_train, epochs=50, batch_size=72, validation_data=(vec_X_test, vec_y_test), verbose=0, shuffle=False)

In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.legend()
plt.show()

In [ ]:
yhat = model.predict(vec_X_test)

<h2 style="color:#27469c">5. Display results</h2>

In [ ]:
results_df = pd.DataFrame({'actual': vec_y_test.reshape((vec_y_test.shape[0])), 'predicted': yhat.reshape((yhat.shape[0]))}, index=y_test.index)
results_df = scaler.denormalize_results(results_df)

mabs_error = mean_absolute_error(results_df['actual'], results_df['predicted'])
rmse = np.sqrt(mean_squared_error(results_df['actual'], results_df['predicted']))
print('Mean Absolute Error: %.3f' % mabs_error)
print('RMSE: %.3f' % rmse)

fig = px.line(results_df, x=results_df.index, y=["actual", "predicted"])
fig.show()

In [ ]:
model.summary()
model.save('../models_saved/kissam1_summer_lstm/')